In [84]:
import wfc
from wfc import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from lasagna.imports import *
import lasagna.io
j = lasagna.config.j = start_client()
dr = ((0, 1),)*100
def show(*args, **kwargs):
    lasagna.io.show_IJ(*args, display_ranges=dr, **kwargs)

In [267]:
N = 3
FMX = N*25
FMY = N*25

a = (1, 0, 1)
b = (0, 1, 0)
c = (1, 1, 1)
d = (0, 0, 0)

T = rotate((c,b,b), 4)
X = rotate((a,b,a), 1)
I = rotate((c,b,c), 2)


initial = [(a,b,a), (b,a,b)]
# initial = T

filename = 'samples/Knot.png'
initial = initial_from_png(filename, N);

model = SimpleTiledModel(N,FMX,FMY,initial)
print 'initial has %d unique tiles' % len(model.initial)
arr = []

initial has 49 unique tiles


In [268]:
%prun -s cumulative do_it()

collapsed (14, 74)
propagation deleted 23344 coefficients in 22194 points; touched 174901 points
collapsed (14, 73)
propagation deleted 1008 coefficients in 256 points; touched 1980 points
collapsed (14, 72)
propagation deleted 1100 coefficients in 255 points; touched 1966 points
collapsed (15, 71)
propagation deleted 284 coefficients in 129 points; touched 1026 points
collapsed (12, 72)
propagation deleted 60 coefficients in 26 points; touched 176 points
collapsed (13, 74)
propagation deleted 2 coefficients in 2 points; touched 10 points
collapsed (12, 73)
propagation deleted 37 coefficients in 17 points; touched 110 points
collapsed (12, 74)
propagation deleted 244 coefficients in 64 points; touched 473 points
collapsed (12, 70)
propagation deleted 78 coefficients in 44 points; touched 342 points
collapsed (12, 69)
propagation deleted 311 coefficients in 129 points; touched 1028 points
collapsed (15, 70)
propagation deleted 179 coefficients in 80 points; touched 637 points
collapsed 

In [86]:
model.wave = np.random.choice([True, False], p=[0.9,0.1], size=model.wave.shape)
# model.wave[0,:,:] = True

In [89]:
%timeit -n1 -r1 model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 2808 points, touched 27036
1 loop, best of 1: 2.56 s per loop


In [99]:
%prun -s cumulative model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 0 points, touched 5512
 

In [151]:
%prun -s cumulative model.update()

collapsed (2, 6)
propagation deleted 18 coefficients in 11 points; touched 63 points
 

In [263]:
arr = []
def do_it():
    global arr
    for i in range(30000):
        done = model.update()
        if done:
            break
        if (i % 20) == 0:
            arr += [model.get_superposition()]
    print i
    return arr

In [221]:
model.wave.sum(axis=0)

array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       ..., 
       [46, 45, 45, ..., 45, 45, 46],
       [46, 45, 45, ..., 45, 45, 46],
       [47, 46, 46, ..., 46, 46, 47]])

In [266]:
show(model.get_superposition(), 'superposition')
# show(model.get_entropy(), 'entropy')
show(model.wave.sum(axis=0), 'wavesum')
# show(model.initial, 'initial')

Cast float64 to float32
Cast int64 to int16 without loss


In [261]:
show(arr)

Cast float64 to float32


In [269]:
arr = np.array(arr)
skimage.external.tifffile.imsave('Knot_run4.tif', arr.astype(np.float32))